In [1]:
import os

import pandas as pd
import numpy as np
import scipy.io as sio
from config import data_dir, data_dict

In [2]:
# 删除os.path.join(data_dir, 'processed_data')下所有csv文件
for i in os.listdir(os.path.join(data_dir, 'processed_data')):
    os.remove(os.path.join(data_dir, 'processed_data', i))

In [3]:
# 样本长度
sample_length = 512

for k,v in data_dict.items():
    # 读取数据
    data = sio.loadmat(os.path.join(data_dir, v))
    # 取出包含'DE_time'的key
    key = [i for i in data.keys() if 'DE_time' in i]
    # 取出该k对应的v
    data_time = data[key[0]]
    # 480000/1024 = 468 
    # 将data_time分割成为468个样本，每个样本为1024个数据点
    num_samples = int(len(data_time) / sample_length)
    for i in range(num_samples):
        data_time_sample = data_time[i*sample_length:(i+1)*sample_length]
        # 对data_time_sample做傅里叶变换
        # data_time_sample = np.fft.fft(data_time_sample)
        # 将data_time_sample保存为csv文件
        df = pd.DataFrame(data_time_sample.reshape(1, -1))
        df.to_csv(os.path.join(data_dir, 'processed_data', f'{k}_{i}.csv'), index=False)


In [4]:
df_lst = []
num = 0
# 读取os.path.join(data_dir, 'processed_data')下所有csv文件
for i in os.listdir(os.path.join(data_dir, 'processed_data')):
    df = pd.read_csv(os.path.join(data_dir, 'processed_data', i))
    df['label'] = i.split('_')[0]
    df_lst.append(df)
    num += 1
# 合并
df_all = pd.concat(df_lst, ignore_index=True)
print(num)

9496


In [5]:
df_all.to_parquet(os.path.join(data_dir, 'all.parquet'))